In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Load Data

In [96]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
path_positive_images_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"chips_positive")
path_to_verified_sets = os.path.join(parent_directory, path_to_complete_dataset,"verified//verified_sets")

## Tile names and urls for all tiles apart of the dataset (complete array)

In [3]:
tile_names_tile_urls_complete_array = fc.add_formatted_and_standard_tile_names_to_tile_names_time_urls(tile_names_tile_urls_complete_array)
print(tile_names_tile_urls_complete_array.shape)

tile_names_tile_urls_complete_array_unique_standard_tile_names, tile_names_tile_urls_complete_array_unique_formatted_tile_names = fc.unique_formatted_standard_tile_names(tile_names_tile_urls_complete_array)
#Save unique standard tile names
np.save("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy", tile_names_tile_urls_complete_array_unique_standard_tile_names)
#pd.DataFrame(np.load("image_download_azure/tile_name_tile_url_complete_array.npy")).to_csv("image_download_azure/tile_name_tile_url_complete_array.csv")

(2610, 4)
unique formatted tile names (2540, 4)
unique standard tile names (2439, 4)


In [4]:
#Move already downloaded tiles to completed dataset
fc.move_tiles_of_verified_images_to_complete_dataset(tile_img_annotation, path_to_tiles_folder_complete_dataset, path_to_verified_sets)

# Make a list of the tiles moved to completed dataset
tiles_downloaded_with_ext_list, tiles_downloaded_without_ext_list = fc.tiles_in_complete_dataset(path_to_tiles_folder_complete_dataset)

Number of tifs to be moved 0


In [6]:
fc.download_tiles_of_verified_images(path_positive_images_complete_dataset, tiles_downloaded, tile_names_tile_urls_complete_array)

1044

In [ ]:
destination_filenames = []
for tile in tile_names_to_download:
    ### download the tiles if they are not in the tiles folder 
    tile_name = tile.split("_",4)[4] #+ ".tif" #get the name of the tile that is to be downloaded
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile_name in string]  #check if the tile name is contained in the string of complete arrays
    tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] # get tile url
    print(tile_name)
    ### Download tile
    destination_filenames.append(ap.download_url(tile_url, path_to_tiles_folder_complete_dataset,
                                                         progress_updater=ap.DownloadProgressBar()))

# Rename Tiles 
for destination_filepath in destination_filenames: 
    tile_dir = destination_filepath.rsplit("\\",1)[0]
    tile_name = destination_filepath.rsplit("\\",1)[1]
    tile_name_split = tile_name.split('_')
    new_tile_path = os.path.join(tile_dir, tile_name_split[6]+'_'+tile_name_split[7]+'_'+tile_name_split[8]+'_'+tile_name_split[9]+''+  \
                                            tile_name_split[10]+'_'+tile_name_split[11]+'_'+tile_name_split[12]+'_'+tile_name_split[13]+'_'+  \
                                            tile_name_split[14]+'_'+tile_name_split[15].split(".")[0]+".tif")
    
    if os.path.isfile(new_tile_path):
        print('Bypassing download of already-downloaded file {}'.format(os.path.basename(new_tile_path)))

    else:
        os.rename(destination_filepath, new_tile_path)

['m_3008816_se_16_060_20191115.tif', 'm_3008816_se_16_060_20191115.tif']
Bypassing download of already-downloaded file m_3008816_se_16_060_20191115.tif
['m_3008824_ne_16_060_20191115.tif', 'm_3008824_ne_16_060_20191115.tif']
Bypassing download of already-downloaded file m_3008824_ne_16_060_20191115.tif
['m_3008832_sw_16_060_20191115.tif']
Bypassing download of already-downloaded file m_3008832_sw_16_060_20191115.tif
['m_3008839_se_16_060_20191115.tif']
Bypassing download of already-downloaded file m_3008839_se_16_060_20191115.tif
['m_3108641_se_16_060_20191102.tif']
Bypassing download of already-downloaded file m_3108641_se_16_060_20191102.tif
['m_3108642_ne_16_060_20190918.tif']
Bypassing download of already-downloaded file m_3108642_ne_16_060_20190918.tif
['m_3108737_ne_16_060_20191110.tif']
Bypassing download of already-downloaded file m_3108737_ne_16_060_20191110.tif
['m_3108737_se_16_060_20191110.tif']
Bypassing download of already-downloaded file m_3108737_se_16_060_20191110.tif


In [112]:
tile_name = 'm_3311805_se_11_060_20180723_20190209.tif'
tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile_name in string]  #check if the tile name is contained in the string of complete arrays
tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]] # get tile url

array([['m_3311805_se_11_060_20180723_20190209.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/ca/2018/ca_60cm_2018/33118/m_3311805_se_11_060_20180723_20190209.tif'],
       ['m_3311805_se_11_060_20180723_20190209.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/33118/m_3311805_se_11_060_20180723_20190209.tif']],
      dtype='<U120')

In [113]:
tile_names_tile_urls_complete_array = np.delete(tile_names_tile_urls_complete_array,
                                                  np.where((tile_names_tile_urls_complete_array[:,0] == tile_name[0]))[0][0],
                                                  axis=0)

In [98]:
tile_names_tile_urls_complete_array.shape

(2609, 2)

In [21]:
for labeled_image_tile_id in tile_id: # the tiles from images 
    if labeled_image_tile_id not in tiles: #the downloaded times 
        #print(labeled_image_tile_id)
        for tile_name_url in tile_names_tile_urls_complete_array[:,]:
            if labeled_image_tile_id == tile_name_url[0].split(".",1)[0]:


['m_3908010_sw_17_060_20180707.tif']

# Determine the pixel location of the image within the tile

In [4]:
#unique positive jpgs (file names with the file extension)
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(parent_directory)

image_characteristics = fc.image_characteristics(path_to_tiles_folder_complete_dataset,unique_positive_jpgs)

image_characteristics.to_csv('image_characteristics.csv')
counterin = 0
counternot = 0 

for unique_jpg in unique_positive_jpgs[:,0]:
    if image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counterin += 1
    if not image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counternot += 1
print(counterin, counternot)

(23441, 2)


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
counter_correct_name = 0
counter_incorrect_name = 0
for jpg in jpgs_without_ext:
    jpg_name_in_image_characteristics = [string for string in image_characteristics['six_digit_chip_name'] if jpg in string] #check if a given tile is in the complete_array 
    if len(jpg_name_in_image_characteristics) >= 1:
        counter_correct_name += 1
    else:
        counter_incorrect_name += 1

In [ ]:
#Identify folders containing positive images